In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [ ]:
y_train = pd.read_csv("UCI HAR Dataset/train/y_train.txt", delim_whitespace=True, header=None).values.ravel()
y_test = pd.read_csv("UCI HAR Dataset/test/y_test.txt", delim_whitespace=True, header=None).values.ravel()
final_df1 = pd.read_csv("geneticalgooutput.csv")
final_df1 = final_df1.iloc[:, :89]
y_combined_df = pd.concat([pd.Series(y_train), pd.Series(y_test)])

In [ ]:
def calculate_similarity(selected_features, ants):
    num_features = len(selected_features)
    if num_features == 0:
        return np.zeros(len(ants[0]))
    similarity = np.sum(ants[:, selected_features], axis=0) / num_features
    similarity = np.pad(similarity, (0, len(ants[0]) - num_features), 'constant')
    return similarity

In [ ]:
def aco_feature_selection(num_features, num_ants, num_iterations_aco, X, y):
    pheromones = np.ones(num_features)
    selected_features = []
    temporary=[i for i in range(10299)]
    ants = X[temporary,:]
    ant_fitness1 = np.array([np.mean(ant) for ant in ants])
    for aco_iteration in range(num_iterations_aco):
        ant_fitness=[]
        random_indices=np.random.choice([i for i in range(10299)],size=num_ants)
        for i in random_indices:
            ant_fitness.append(ant_fitness1[i])

        best_ant_index = np.argmax(ant_fitness)



        if 0 <= best_ant_index < 10299:
            best_ant_position = ants[best_ant_index].copy()

            similarity = calculate_similarity(selected_features, ants)
            for feature_i in range(num_features):
                pheromones[feature_i] += ant_fitness[best_ant_index] * (1 - similarity[feature_i])

            pheromone_probs = np.exp(pheromones) / np.sum(np.exp(pheromones))

            selected_feature = np.random.choice(np.arange(num_features), p=pheromone_probs)
            selected_features.append(selected_feature)  # Increment by 1 to match original feature indices

            print(f"ACO Iteration {aco_iteration + 1}/{num_iterations_aco}: Selected feature {selected_feature}")
        else:
            print("Invalid best_ant_index:", best_ant_index)

    return selected_features

In [ ]:
X = final_df1
y = y_combined_df

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_final = X_scaled[:]

selected_features_aco = aco_feature_selection(X_final.shape[1], num_ants=200, num_iterations_aco=47, X=X_final, y=y)
selected_features_aco

X_final = X_scaled[:,selected_features_aco]

X_final=pd.DataFrame(X_final)
X_final.head(10299)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from niapy.problems import Problem
from niapy.task import Task
from niapy.algorithms.basic import ParticleSwarmOptimization
import torch

In [ ]:
class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=[0] * X_train.shape[1], upper=[1] * X_train.shape[1])
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)


X_scaled = scaler.fit_transform(X_final)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_encoded = np.ravel(y_encoded)
X_tensor = torch.FloatTensor(X_scaled.astype('float32'))
y_tensor = torch.LongTensor(y_encoded).ravel()


device = torch.device("cpu")
X_tensor, y_tensor = X_tensor.to(device), y_tensor.to(device)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

problem = SVMFeatureSelection(X_train, y_train)
task = Task(problem, max_iters=100)
algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
best_features, best_fitness = algorithm.run(task)


selected_features = best_features > 0.5
print('Number of selected features:', selected_features.sum())
print('Selected features:', ', '.join(map(str, np.where(selected_features)[0])))

model_selected = SVC()
model_all = SVC()

model_selected.fit(X_train[:, selected_features], y_train)
print('Subset accuracy:', model_selected.score(X_test[:, selected_features], y_test))

model_all.fit(X_train, y_train)
print('All Features Accuracy:', model_all.score(X_test, y_test))


In [ ]:
X_final_nparray=np.array(X_final)
X_temparr=X_final_nparray[:,selected_features]
X_temp=pd.DataFrame(X_final_nparray[:,selected_features])
X_temp.head(10299)

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [ ]:
label_encoder = LabelEncoder()
y_combined_encoded = label_encoder.fit_transform(y)

X_tensor = torch.FloatTensor(X_temparr)
y_tensor = torch.LongTensor(y_combined_encoded)

device = torch.device("cpu")
X_tensor, y_tensor = X_tensor.to(device), y_tensor.to(device)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class HybridModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(HybridModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.lstm1 = nn.LSTM(input_size=128, hidden_size=hidden_size, batch_first=True, dropout=0.5)
        self.lstm2 = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = torch.relu(self.conv1(x.unsqueeze(1)))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = x.permute(0, 2, 1)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

model1 = HybridModel(input_size=20, hidden_size=64, num_classes=len(label_encoder.classes_)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)

train_losses = []
train_accuracies = []
test_accuracies = []

num_epochs = 200
for epoch in range(num_epochs):
    model1.train()
    total_loss = 0.0
    correct_train_predictions = 0
    total_train_samples = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train_samples += labels.size(0)
        correct_train_predictions += (predicted == labels).sum().item()

    train_epoch_loss = total_loss / len(train_loader)
    train_epoch_accuracy = correct_train_predictions / total_train_samples

    train_losses.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)

    model1.eval()
    correct_test_predictions = 0
    total_test_samples = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model1(inputs)
            _, preds = torch.max(outputs, 1)
            total_test_samples += labels.size(0)
            correct_test_predictions += (preds == labels).sum().item()

    test_epoch_accuracy = correct_test_predictions / total_test_samples

    test_accuracies.append(test_epoch_accuracy)


    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Loss: {train_epoch_loss:.4f}, Train Accuracy: {train_epoch_accuracy:.4f}, "
          f"Test Accuracy: {test_epoch_accuracy:.4f}")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy')
plt.plot(range(1, num_epochs + 1), test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

activity_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']

adjusted_all_labels = [label - 1 for label in all_labels]
adjusted_all_preds = [pred - 1 for pred in all_preds]

conf_matrix = confusion_matrix(adjusted_all_labels, adjusted_all_preds)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=activity_names, yticklabels=activity_names, annot_kws={"size": 10})
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')

plt.xticks(rotation=90)

plt.show()